In [4]:
import pandas as pd
import numpy as np

# NOTA: Para salvar em .xlsx (Excel), o pandas precisa da biblioteca openpyxl.
# Se você receber um erro de "Missing dependency", rode no seu terminal:
# pip install openpyxl

# --- 1. CONFIGURAÇÃO INICIAL ---
# O nome do arquivo que você subiu
# ATUALIZADO: Usei um "r" antes da string (raw string) para o Python entender a \
NOME_ARQUIVO_ENTRADA = r"dados\rd_clean.xlsx"

# Listas para filtrar os totais (O MAIS IMPORTANTE!)
# Adicione qualquer outro valor de "total" que você encontrar
AGRUPAMENTOS_PARA_REMOVER = [
    "TOTAL FM"
]

PUBLICO_PARA_REMOVER = [
    "SEXO AMBOS" # Exemplo de total a ser removido (confirme se existe no seu arquivo!)
    # Adicione outros aqui, ex: "TOTAL IDADE", "TODAS AS CLASSES"
]

# MOVIDO: A definição das condições agora é feita DENTRO do try,
# depois que o 'df' é carregado.

# Colunas que são métricas (para fazer o unpivot/melt)
# Baseado no seu CSV, as colunas são 'opm', 'os', 'alc2', 'tmed'
COLUNAS_METRICAS = [
    'opm',
    'os',
    'alc2',
    'tmed'
]

# Colunas que identificam a linha (todas, menos as métricas)
# A 'publico' vai ser tratada, então não entra aqui
# ATUALIZADO: Removi 'days' desta lista
COLUNAS_ID = [
    'data',
    'agrupamento'
]

print(f"--- Iniciando processamento do arquivo: {NOME_ARQUIVO_ENTRADA} ---")

# --- 2. LEITURA E LIMPEZA INICIAL ---
try:
    # ATUALIZADO: Trocado de read_csv para read_excel
    df = pd.read_excel(NOME_ARQUIVO_ENTRADA, parse_dates=['data'])
    print(f"Arquivo lido com sucesso! Shape inicial: {df.shape}")

    # ATUALIZADO: Removendo a coluna 'days' logo no início
    if 'days' in df.columns:
        df = df.drop(columns=['days'])
        print(f"Coluna 'days' removida. Novo shape: {df.shape}")
    else:
        print("Coluna 'days' não encontrada para remover.")

    # Passo 1: Filtrar TODOS os Totais
    df_filtrado = df[~df['agrupamento'].isin(AGRUPAMENTOS_PARA_REMOVER)]
    df_filtrado = df_filtrado[~df_filtrado['publico'].isin(PUBLICO_PARA_REMOVER)]
    
    print(f"Shape após filtrar totais: {df_filtrado.shape}")
    
    # --- 3. "QUEBRAR" A COLUNA 'publico' ---
    
    # ADICIONADO AQUI: Definindo as condições DEPOIS que 'df_filtrado' existe
    CONDICOES_SEGMENTO = [
        (df_filtrado['publico'].str.startswith("CLASSE ", na=False)),
        (df_filtrado['publico'].str.startswith("IDADE ", na=False)),
        (df_filtrado['publico'].str.startswith("SEXO ", na=False)),
        (df_filtrado['publico'].str.startswith("ZONA ", na=False))
    ]
    ESCOLHAS_SEGMENTO = [
        "Classe",
        "Idade",
        "Sexo",
        "Zona"
    ]
    
    # Criando a coluna "Tipo de Segmento" (Classe, Idade, Sexo, Zona)
    # Usamos np.select que é super rápido!
    df_filtrado['Tipo de Segmento'] = np.select(
        CONDICOES_SEGMENTO, 
        ESCOLHAS_SEGMENTO, 
        default='Outro'
    )
    
    # Limpando a coluna 'publico' para virar 'Segmento'
    # Vamos usar regex para remover todos os prefixos de uma vez
    regex_pattern = r'^(CLASSE |IDADE |SEXO |ZONA )'
    df_filtrado['Segmento'] = df_filtrado['publico'].str.replace(
        regex_pattern, 
        '', 
        regex=True
    ).str.strip()

    # --- 4. TRANSFORMAR MÉTRICAS EM "FORMATO LONGO" (Unpivot/Melt) ---
    
    # Adicionamos as novas colunas de segmento às colunas de ID
    colunas_id_final = COLUNAS_ID + ['Tipo de Segmento', 'Segmento']
    
    df_long = df_filtrado.melt(
        id_vars=colunas_id_final,
        value_vars=COLUNAS_METRICAS,
        var_name='Metrica',      # Nova coluna com os nomes (opm, os, etc)
        value_name='Valor'       # Nova coluna com os valores numéricos
    )
    
    print(f"Shape após 'unpivot' (melt): {df_long.shape}")

    # --- 5. LIMPEZA FINAL ---
    
    # Substituir 'null' (NaN) por 0 na coluna Valor
    df_long['Valor'] = df_long['Valor'].fillna(0)
    
    # ATUALIZADO: Removi a seção opcional de drop no final,
    # pois já fizemos isso no começo.

    print("--- Processamento Concluído! ---")
    print("\n5 primeiras linhas da tabela final transformada:")
    print(df_long.head())

    print("\nExemplo de valores criados:")
    print("Tipos de Segmento:", df_long['Tipo de Segmento'].unique())
    print("Métricas:", df_long['Metrica'].unique())

    # Opcional: Salvar o resultado
    # ATUALIZADO: Trocando de CSV para XLSX!
    NOME_ARQUIVO_SAIDA = "dados_prontos_para_previsao.xlsx"
    df_long.to_excel(NOME_ARQUIVO_SAIDA, index=False)
    print(f"\nArquivo salvo em: {NOME_ARQUIVO_SAIDA}")

except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado! Verifique o nome: {NOME_ARQUIVO_ENTRADA}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")




--- Iniciando processamento do arquivo: dados\rd_clean.xlsx ---
Arquivo lido com sucesso! Shape inicial: (19072, 7)
Coluna 'days' não encontrada para remover.
Shape após filtrar totais: (17025, 7)
Shape após 'unpivot' (melt): (68100, 6)
--- Processamento Concluído! ---

5 primeiras linhas da tabela final transformada:
        data   agrupamento Tipo de Segmento   Segmento Metrica  Valor
0 2025-09-01  BAND NEWS FM             Sexo   FEMININO     opm    0.0
1 2025-09-01  BAND NEWS FM             Sexo  MASCULINO     opm    0.0
2 2025-09-01  BAND NEWS FM           Classe         AB     opm    0.0
3 2025-09-01  BAND NEWS FM           Classe          C     opm    0.0
4 2025-09-01  BAND NEWS FM           Classe         DE     opm    0.0

Exemplo de valores criados:
Tipos de Segmento: ['Sexo' 'Classe' 'Idade' 'Zona']
Métricas: ['opm' 'os' 'alc2' 'tmed']

Arquivo salvo em: dados_prontos_para_previsao.xlsx
